# ABM simulators
In this notebook we define the simulator of the bounded confidence model fitting the formalism of `blackit` package.

This will be used for MSM calibration of $\epsilon$ parameter.
MSM is a simulation-based technique. 
- the parameter is sampled under a sampling scheme
- the simulation is run with the selected parameter
- a distance measure is compute between the observed time series and the simulated time series
- the previous steps are repeated a given number of times
- the estimate of the parameter is the one that minimized the distance measure between observed and simulated traces

In [1]:
import numpy as np
import sys
sys.path += ["../src"]
import torch
import simulator_opinion_dynamics as sod
from scipy.special import expit as sigmoid

### Full Observed BCM

In [2]:
class FBCM_simulator():
    def __init__(self, X0, edges, N, mu, real_epsilon, rho = 16, seed = 1):
        self.X0 = X0
        self.N = N
        self.edges = edges.clone().long()
        _, self.edge_per_t, _ = edges.shape
        self.mu = mu
        self.real_epsilon = real_epsilon
        self.rho = rho
        self.seed = seed
        # self.ts is the time series of the proportion of signs s (the 3rd column of edges)
        # the calibrator will compare the real ts with the simulated ts
        self.ts = np.atleast_2d(edges[:,:,2].sum(dim = 1) / edge_per_t).T
    
    def simulate_ts(self, theta, N = 200, seed = None):
        epsilon, = theta
        mean_s_pred = []
        
        
        if seed != None:
            np.random.seed(self.seed)
        
        X_t = self.X0.clone()
        
        diff_X = self.X0[:,None] - self.X0[None,:]
        
        T, edge_per_t, _ = self.edges.size()
        
        for t in range(T):
            u,v,s_obs = self.edges[t].T
            
            s_pred = (torch.rand(self.edge_per_t) < torch.sigmoid(self.rho * (epsilon - torch.abs(diff_X[u,v])))) + 0.
            mean_s_pred.append(s_pred.mean())
            
            X_t, diff_X = sod.opinion_update_BC(diff_X, X_t, self.edges[t], self.N, (epsilon, self.mu, self.rho))
            
        return np.atleast_2d(mean_s_pred).T
    

In [3]:
N, T, edge_per_t = 100, 256, 4
evidences_per_t = 4
epsilon, mu, rho = 0.35, 0.4, 16

X, edges, evidences = sod.simulate_BC(N, T, edge_per_t, evidences_per_t, (epsilon, mu, rho), seed = 34945)

In [4]:
fbcm_simulator = FBCM_simulator(X[0], edges, N, mu, epsilon, rho)

In [5]:
s_simulated = fbcm_simulator.simulate_ts(theta = [0.2])

### Partially Observed BCM

In [6]:
class PBCM_simulator():
    def __init__(self, X0, edges, N, mu, real_epsilon, rho = 16, seed = 1, sample_all_edges = True):
        self.X0 = X0
        self.N = N
        self.edges = edges.clone().long()
        T_, self.edge_per_t, _ = edges.shape
        self.T = T_ + 1
        self.mu = mu
        self.real_epsilon = real_epsilon
        self.rho = rho
        self.seed = seed
        self.ts = np.atleast_2d(edges[:,:,2].sum(dim = 1) / self.edge_per_t).T
        self.s_edges = edges[:,:,2]
        
        self.sample_all_edges = sample_all_edges
    
    def simulate_ts(self, theta, N = 200, seed = None):
        epsilon, = theta
        mean_s_pred = []
        
        
        if seed != None:
            np.random.seed(self.seed)
        
        X_t = self.X0.clone()
        diff_X = self.X0[:,None] - self.X0[None,:]
        
        
        for t in range(self.T - 1):
            edges_t = self.edges[t]
            random_edges_t = torch.randint(low = 0, high = self.N, size = (self.edge_per_t, 2))#, dtype = torch.float32)
            mask = edges_t[:,2] == 0
            edges_t[mask, :2] = random_edges_t[mask]
            
            u,v,s_obs = edges_t.T
            
            if self.sample_all_edges:
                u,v = random_edges_t.T
            else:
                u,v = edges_t.T[:2,:]

            s_pred = (torch.rand(self.edge_per_t) < torch.sigmoid(self.rho * (epsilon - torch.abs(diff_X[u,v])))) + 0.
            mean_s_pred.append(s_pred.mean().item())
            
            X_t, diff_X = sod.opinion_update_BC(diff_X, X_t, self.edges[t], self.N, (epsilon, self.mu, self.rho))
            
        return np.atleast_2d(mean_s_pred).T

    

In [7]:
pbcm_simulator = PBCM_simulator(X[0], edges, N, mu, epsilon, rho)

s_simulated = pbcm_simulator.simulate_ts(theta = [0.2])

### Noisy observations BCM
In this case, the opinions are latent. 
So, at each step, we need to sample uniformly the opinions of the agents. 

We want to compare the outcomes of the interactions (s) and the proxies of the opinions.
So, the observed and simulated time series we'll compare have three dimensions. (1) the signs of the interactions (s), (2) the mean of the sampled proxies of the opinions, (3) the variance of the sampled proxies of the opinions.

In [8]:
class NBCM_simulator():
    def __init__(self, N, edges, evidences, mu, real_epsilon, evidence_distribution = "bernoulli", 
                 sum_ab = 1, rho = 16, seed = 1):
        T, edge_per_t, _ = edges.shape
        self.evidence_distribution = evidence_distribution
        self.evidences_per_t = len(evidences[0][0])
        self.T = T
        self.edge_per_t = edge_per_t
        self.N = N
        self.evidences = evidences
        self.edges = edges.clone().long()
        self.sum_ab = sum_ab
        self.mu = mu
        self.real_epsilon = real_epsilon
        self.rho = rho
        self.seed = seed
        #the time series used in the comparison takes into account the signs, the mean and the variances of the evidences (proxies of the opinions)
        self.ts = np.atleast_2d([(torch.mean(evidences[t][1]).item(),
                                  torch.var(evidences[t][1]).item(),
                                  edges[t,:,2].sum() / edge_per_t) for t in range(T)])
    
    def simulate_ts(self, theta, N = 200, seed = None):
        epsilon, = theta
        
        mean_ev_pred, var_ev_pred, mean_s_pred = [], [], []
        
        if seed != None:
            np.random.seed(self.seed)
        
        X0 = torch.rand(self.N)
        X_t = X0.clone()
        diff_X = X0[:,None] - X0[None,:]
        
        for t in range(self.T):
            u,v,s_obs = self.edges[t].T
            
            s_pred = (torch.rand(self.edge_per_t) < torch.sigmoid(self.rho * (epsilon - torch.abs(diff_X[u,v])))) + 0.
            X_t, diff_X = sod.opinion_update_BC(diff_X, X_t, self.edges[t], self.N, (epsilon, self.mu, self.rho))
            
            u_evidences = self.evidences[t][0]
            
            if self.evidence_distribution == "beta":
                a,b = self.sum_ab * (X_t[u_evidences]), self.sum_ab * (1 - (X_t[u_evidences]))
                evidences_t = Beta(torch.Tensor(a), torch.Tensor(b)).sample()
                mean_ev_pred.append(torch.mean(evidences_t).item())
                var_ev_pred.append(torch.var(evidences_t).item())
            
            if self.evidence_distribution == "bernoulli":
                evidences_t = (torch.rand(self.evidences_per_t) < X_t[u_evidences]) + 0.
                mean_ev_pred.append(torch.mean(evidences_t))
                var_ev_pred.append(torch.var(evidences_t))
            
            
            mean_s_pred.append(s_pred.mean())
        
        return np.atleast_2d([mean_ev_pred, var_ev_pred, mean_s_pred]).T

In [9]:
nbcm_simulator = NBCM_simulator(N, edges, evidences, mu, epsilon)

ts_simulated = nbcm_simulator.simulate_ts(theta = [0.2])